# 李鲁鲁老师的Python课程 第七课 

类的后续内容，以及一个在程序中引入信息学竞赛算法的例子

还记得上一节课的小汽车的例子吗上一节课我们的小汽车是随机移动的

在这节课的时候，我们希望小汽车能够按照我们的想法去移动

```
在images/car.jpg存储了一张小汽车的照片，

我希望实现一个pygame中的Car类

这个类可以用( x, y, speed = 5, size = 100)来进行初始化

初始化后current_x, current_y分别为x,y， 为汽车中的位置

并且可以用set_target( x, y ) 来设置目标的位置

并且Car类有update()方法，每次会以speed的速度向目标移动，如果移动到了目标位置(speed以内)，则在目标位置停止移动

另外还有render(screen)方法可以在pygame中进行渲染

请帮我实现
```


In [ ]:
# 不要在notebook运行这个代码，在pygame_codes/Car.py中运行

import pygame
import math

class Car:
    def __init__(self, x, y, speed=5, size=100):
        self.current_x = x
        self.current_y = y
        self.speed = speed
        self.size = size
        self.target_x = x
        self.target_y = y
        self.image = pygame.image.load("images/car.jpg")
        self.image = pygame.transform.scale(self.image, (size, size))
    
    def set_target(self, x, y):
        self.target_x = x
        self.target_y = y

    def update(self):
        dx = self.target_x - self.current_x
        dy = self.target_y - self.current_y
        distance = math.hypot(dx, dy)
        
        if distance < self.speed:
            self.current_x = self.target_x
            self.current_y = self.target_y
        else:
            angle = math.atan2(dy, dx)
            self.current_x += self.speed * math.cos(angle)
            self.current_y += self.speed * math.sin(angle)

    def render(self, screen):
        screen.blit(self.image, (self.current_x, self.current_y))

if __name__ == "__main__":

    # 初始化 Pygame
    pygame.init()

    # 设置屏幕大小
    screen = pygame.display.set_mode((800, 600))

    # 设置窗口标题
    pygame.display.set_caption("Car Simulation")

    # 实例化一个Car对象
    car = Car(100, 100)

    # 运行主循环
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # 更新汽车位置
        car.update()

        # 绘制屏幕
        screen.fill((255, 255, 255))
        car.render(screen)
        pygame.display.flip()

        # 设置目标位置（测试用，实际应用中可以根据具体需求设置）
        car.set_target(400, 300)

        # 控制帧率
        pygame.time.Clock().tick(30)

    pygame.quit()


这个代码老师放在了pygame_codes/Car.py中，

可以看到ChatGPT帮我在100*100中心渲染了一辆车，并且给他移动到了屏幕中间的位置。

# 四辆向对方走的小车

我突然想到一道经典的数学问题

那就是在正方形上四个角放置四个点，然后让这四个点向对方走，直到相遇

我们其实可以从car.current_x和car.current_y读到每一辆汽车各自的位置

如果我们让每一辆车都向对方移动，那会怎么样呢？

```
我已经实现了Car类，可以from Car import Car来载入

Car类支持用x,y, speed(默认是5)，size(默认是100)来初始化，并且支持set_target(x,y)来设定要走向的目标，update()进行更新以及render(screen)来进行渲染

我想在600*600的屏幕的4个角上，渲染4个50*50的汽车，他们每次都会用car.current_x和car.current_y读取前一个index的车的位置

然后设定前一个车为目标，请用python为我实现。
```

---

这里我们要说一个“连续聊天”的问题

因为不少ChatGPT的支持这样的连续聊天了

连续聊天也就是指我在开发Car.py这个类的时候，我进一步提出后续的需求

我发现在使用ChatGPT-4o的情况下，如果连续聊天，是可以一次性把这个需求写对的

我把这个版本放在了pygame_codes/7_CarRun_ChatGPT.py中了

---

当然，如果你一次性就写对就没什么问题了，但是李鲁鲁老师在尝试直接把我们的需求prompt输入到ChatGPT中得到的程序的时候

我们得到了 pygame_codes/7_CarRun.py

这个代码是可以正常运行的，但是运行的时候我们发现了和我们预期不一样

这里我们先注意到了

```python
    # 重新设定目标
    for i in range(1, len(cars)):
        cars[i].set_target(cars[i-1].current_x, cars[i-1].current_y)
```

这段话没有让第0辆车去追第3辆车。于是我们在这个下面补了一句话

```python
    # 李鲁鲁老师手动添加
    cars[0].set_target(cars[-1].current_x, cars[-1].current_y)
```

这句话让0号车有了目标动了起来，但是我们发现仍然不及预期

这是由于

```python
# 创建四个汽车对象，放置在屏幕的四个角
cars = [
    Car(x=0, y=0, size=50),
    Car(x=screen_size-50, y=0, size=50),
    Car(x=0, y=screen_size-50, size=50),
    Car(x=screen_size-50, y=screen_size-50, size=50),
]
```

在初始化的时候，我们的需求prompt并没有明确说明我们需要这几辆车需要以顺时针方向出现

所以这4个cars并没有用顺时针排列

我们发现只要交换最后两辆车的位置就可以正确运行了

---

这个例子其实挺重要的

- 我们的prompt有可能有些需求没有说明白，比如刚才那个需求里面的“顺时针”和“前一个”其实就是相对模糊的，这个时候有可能会导致程序和我们的预期不一致

- 但是很多时候你可以通过轻微地调整程序，来最终获得期望的结果

- 另一方面，保持连续对话，有可能会获得更好的效果。语言模型会自己根据前面需求的上下文，来更精准地对程序进行实践。

---

# 回到数数游戏

让我们回到一个之前的例子，

记得在第五课的时候，我们写过一个 汽车游戏吗？

代码放在了pygame_codes/5_car_0.py

但是这里面按键按下的时候，汽车都是突然出现的。

我们有办法把汽车用动画进行渐变的移动吗？

下面是prompt

```
我已经实现了一个python函数random_points(N, min_distance, screen_width, screen_height)

这个函数可以输出一个list of tuple，其中每个tuple表示一个点的坐标，并且保证任意两个点之间的距离大于min_distance

并且我实现了一个Car类

我已经实现了Car类，可以from Car import Car来载入

Car类支持用x,y, speed(默认是5)，size(默认是100)来初始化，并且支持set_target(x,y)来设定要走向的目标，update()进行更新以及render(screen)来进行渲染

---

现在我希望用pygame实现一个游戏

维护一个cars的list来表示当前的汽车

我希望1-9数字键按下的时候，会重新生成对应数量的汽车位置

并且将汽车位置设置到对应的屏幕位置，多余的汽车，将目标设置到屏幕外

请帮我实现
```

----

这里我在7_car_with_num_ChatGPT_0.py 放置了第一个版本

但是我们想做一些修改

```
在regenerate_cars的时候

不要直接clear整个cars，如果cars数量超过num_cars，则从尾部把多余的cars的目标设置到屏幕外侧

如果num_cars多于cars，则用现在的方案生成对应的汽车

并且更新现存的汽车的目标位置
```

运行pygame_codes/7_car_with_num_ChatGPT_1.py查看修改后的效果

# 改进作业

- 边界问题
- 最小二分匹配
- 播放声音

## 小结

- 进一步使用面向对象的方法去编写pygame游戏

- 可以设置target位置的Car类以及pygame的动画

- 但是作为一门入门的课程，继承之类的方法我们并没有着重介绍

## 去准备智谱的api

对了，在这节课，同学们要记得检查有没有注册智谱的api

登录[https://open.bigmodel.cn/](https://open.bigmodel.cn/)

通过个人验证之后，可以获得免费赠送的几百万的token

然后带年纪API密钥，

赋值里面的APIkey，（如果没有的话添加新的key）

类似这样

```
a6e3215e16safdkja5lksdc65bf563.23qs209d2cnYcG
```

想办法把key放到一个特定txt的文件中。

这样我们在下一节课就可以顺利进行了。